In [ ]:
# !pip install glob2

In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import gensim.downloader
import numpy as np

from pathlib import Path

import glob

# 1. Лемматизация

Дальнейшая разметка строится на предположении, что первое существительное в заголовке столбца отражает ту сущность, которая в нем содержится

In [2]:
import spacy

nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

def lemmatize_word(word: str) -> str|None:
    for token in nlp(sequence):
        if token.pos_ == "NOUN":
            return token.lemma_
    return None

In [3]:
number_of_chunks = len(glob.glob("./cleansed/*"))

for chunk_id in range(number_of_chunks):
    if not Path(f"./lemmatized/lemmatized_{chunk_id}.csv").exists():
        df_cleansed = pd.read_csv(f"./cleansed/df_cleansed_{chunk_id}.csv", sep=";")
    
        df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lemmatize_word)
        df_cleansed.to_csv(f"./lemmatized/lemmatized_{chunk_id}.csv", sep=";", index=False)

In [4]:
df_list = []

for i in range(number_of_chunks):
    df_list.append(
        pd.read_csv(f"lemmatized/lemmatized_{i}.csv", sep=";")
    )
df = pd.concat(df_list, axis=0)
df

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
337142,dataset/Dataset/data/35185/table_5_meta.json,2,дата
337143,dataset/Dataset/data/35185/table_5_meta.json,3,дата
337144,dataset/Dataset/data/35185/table_5_meta.json,4,дата
337145,dataset/Dataset/data/35185/table_5_meta.json,5,дата


In [5]:
unique_col_name_df = df.drop_duplicates(subset=["column_name"])
unique_col_name_df

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
333591,dataset/Dataset/data/561562/table_2_meta.json,3,подъм
335746,dataset/Dataset/data/6446239/table_0_meta.json,5,секеи
336733,dataset/Dataset/data/956099/table_0_meta.json,0,ибаге
336734,dataset/Dataset/data/956099/table_0_meta.json,1,невадос


In [6]:
unique_col_name_df.sort_values("column_name")["column_name"][:100]

114170                    аа
140548                   аай
73017                    аал
144824                   аах
187760                   аба
                 ...        
103534            автоматика
41468              автомашин
125920    автомобилестроение
922               автомобиль
28561      автономностьсуток
Name: column_name, Length: 100, dtype: object

# 2. Чтение семантических типов

Изначальные типы изменил, которые состояли из > 1 слова для w2v

In [7]:
sem_types = pd.read_csv("sem_types.txt", sep=";", header=None, names=["labels"])
sem_types = sem_types.drop_duplicates()
sem_types["labels"] = sem_types["labels"].apply(lambda x: str(x).lower())

# 3. Разметка
Для кластеризации удаляем столбцы, для которых не было получено ембеддингов

In [8]:
ru_model = gensim.downloader.load("word2vec-ruscorpora-300")

In [9]:
def get_most_similar(word: str) -> str|None:
    """Need to split afterwards because returns (label cos.sim.value)
    """

    max_similarity = 0
    max_similar_label = None
    
    try:
        ru_model.__getitem__([f"{word}_NOUN"])[0]
    except KeyError:
        return f"{max_similar_label} {max_similarity}"

    for label in sem_types["labels"]:
        try:
            similarity = ru_model.similarity(f"{label}_NOUN", f"{word}_NOUN")
        except KeyError:
            continue

        if similarity > max_similarity:
            max_similarity = similarity
            max_similar_label = label

    return f"{max_similar_label} {max_similarity}"

In [10]:
if not Path("./labelled/labelled_1.csv").exists():
    df_list = []
    for i in range(number_of_chunks):
        df_list.append(
            pd.read_csv(f"./lemmatized/lemmatized_{i}.csv", sep=";", on_bad_lines="warn")
        )
    df = pd.concat(df_list, axis=0)

    # df = df.sample(frac=0.001)
    
    df["label"] = df["column_name"].progress_apply(get_most_similar)
    df.to_csv("./labelled/labelled_1.csv", sep=";", index=False)
else:
    df = pd.read_csv("./labelled/labelled_1.csv", sep=";")

df

,table_id,column_id,column_name,label
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт,сингл 0.5395997762680054
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция,None 0
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда,команда 1.0000001192092896
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки,мотоциклист 0.3064345717430115
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год,год 0.9999999403953552
...,...,...,...,...
3371473,dataset/Dataset/data/35185/table_5_meta.json,2,дата,дата 1.0
3371474,dataset/Dataset/data/35185/table_5_meta.json,3,дата,дата 1.0
3371475,dataset/Dataset/data/35185/table_5_meta.json,4,дата,дата 1.0
3371476,dataset/Dataset/data/35185/table_5_meta.json,5,дата,дата 1.0


In [11]:
df[["label", "similarity"]] = df["label"].str.strip().str.split(" ", n=1, expand=True)
df

,table_id,column_id,column_name,label,similarity
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт,сингл,0.5395997762680054
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция,None,0
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда,команда,1.0000001192092896
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки,мотоциклист,0.3064345717430115
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год,год,0.9999999403953552
...,...,...,...,...,...
3371473,dataset/Dataset/data/35185/table_5_meta.json,2,дата,дата,1.0
3371474,dataset/Dataset/data/35185/table_5_meta.json,3,дата,дата,1.0
3371475,dataset/Dataset/data/35185/table_5_meta.json,4,дата,дата,1.0
3371476,dataset/Dataset/data/35185/table_5_meta.json,5,дата,дата,1.0


In [16]:
df["table_id"] = df["table_id"].str.strip().str.split("_meta.json", n=1, expand=True)[0]
df

,table_id,column_id,column_name,label,similarity
0,dataset/Dataset/data/7779522/table_0,0,чарт,сингл,0.5395997762680054
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция,None,0
2,dataset/Dataset/data/2863377/table_11,1,команда,команда,1.0000001192092896
3,dataset/Dataset/data/2863377/table_11,9,очки,мотоциклист,0.3064345717430115
4,dataset/Dataset/data/4727026/table_0,0,год,год,0.9999999403953552
...,...,...,...,...,...
3371473,dataset/Dataset/data/35185/table_5,2,дата,дата,1.0
3371474,dataset/Dataset/data/35185/table_5,3,дата,дата,1.0
3371475,dataset/Dataset/data/35185/table_5,4,дата,дата,1.0
3371476,dataset/Dataset/data/35185/table_5,5,дата,дата,1.0


## Выбор порогового значения

|порог|кол-во заголовков, схожесть >= порог|
|-----|------------------------------------|
|0.1|2.993.652|
|0.2|2.993.550|
|0.3|2.968.070|
|0.4|2.754.081|
|0.5|2.186.338|
|0.6|1.659.870|
|0.7|1.505.095|
|0.8|1.435.068|
|0.9|1.427.649|
|1.0|901.335|

P.s. Значения >= 1.0 за счет погрешности чисел с палвающей точкой

In [12]:
for i in range(1, 10+1):
    sim = (df["similarity"].apply(float) >= i / 10).sum()
    print(f"{i / 10} : {sim}")

0.1 : 2993652
0.2 : 2993550
0.3 : 2968070
0.4 : 2754081
0.5 : 2186338
0.6 : 1659870
0.7 : 1505095
0.8 : 1435068
0.9 : 1427649
1.0 : 901335


In [17]:
threshold = 0.77

df_threshold = df[df["similarity"].astype(float) > threshold]
df_threshold.sort_values("similarity").head(10)

,table_id,column_id,column_name,label,similarity
705762,dataset/Dataset/data/6922823/table_0,0,переулок,улица,0.7720689177513123
705763,dataset/Dataset/data/6922823/table_0,1,переулок,улица,0.7720689177513123
2832447,dataset/Dataset/data/8487286/table_6,1,бомбардировщик,самолет,0.7729793190956116
949618,dataset/Dataset/data/5252496/table_1,3,бомбардировщик,самолет,0.7729793190956116
1186337,dataset/Dataset/data/4496195/table_2,1,бомбардировщик,самолет,0.7729793190956116
1744297,dataset/Dataset/data/6848959/table_3,3,бомбардировщик,самолет,0.7729793190956116
3191290,dataset/Dataset/data/4496192/table_2,1,бомбардировщик,самолет,0.7729793190956116
1773191,dataset/Dataset/data/6247054/table_3,3,бомбардировщик,самолет,0.7729793190956116
1336802,dataset/Dataset/data/6848960/table_3,3,бомбардировщик,самолет,0.7729793190956116
2154787,dataset/Dataset/data/5267091/table_3,3,бомбардировщик,самолет,0.7729793190956116


In [18]:
df_threshold.to_csv(f"labelled_threshold_{threshold}.csv", sep=";", index=False)